In [5]:
import numpy as np
from scipy.optimize import minimize
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.spatial import ConvexHull


In [ ]:
def FastRF_3D_compoundBar_thermalLoads(Uk, u_pe, Lk1, flag_thermalAnalysisType, prop):
    
    '''
    
    Retuns critical RF for a set of loads and a performance envelope    
    
    Inputs
    -------
    Uk (array): characteristic loads in cartisian coordinates 3D space
    u_pe (array): envelope points in 3D space
    
    Outputs
    -------
    RF_pe: Critical RFs  
    
    Dependency
    ----------
    NA
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 07/03/2018
    
    '''
    RF_ana = []
    RFind = []
    for p in Uk:
        rfmin, rfind = check_r_compoundBar_thermalLoads(p, Lk1, flag_thermalAnalysisType, prop )  
        RF_ana.append(rfmin)

    #Convert list to array
    RF_ana = np.asarray(RF_ana)

    #critical RF from envelope
    RF_pe = Fast_RF_estimation_3D(Uk,u_pe)
    
    return RF_ana, RF_pe

In [ ]:
def Fast_RF_estimation_3D(Uk,u_pe):
    
    '''
    Retuns critical RF for a set of loads and a performance envelope    
    
    Inputs
    -------
    Uk (array): characteristic loads in cartisian coordinates 3D space
    u_pe (array): envelope points in 3D space
    
    Outputs
    -------
    RF_pe: Critical RFs  
    
    Dependency
    ----------
    NA
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 07/03/2018
    '''
    I = calculate_intersection_3D(Uk,u_pe)

    r_allowable, temp_t,temp_p = cart2sph(I[:,0],I[:,1],I[:,2])
    r_actual, temp1_t,temp2_p = cart2sph(Uk[:,0],Uk[:,1],Uk[:,2])
    RF_pe = (r_allowable/r_actual).T

    return RF_pe

In [ ]:
def calculate_intersection_3D(loads,u_pe):
    
    '''
    Retuns intersection of the point with a plane formed by 3 points in a 3D space
    Notations used is from wiki https://www.wikiwand.com/en/Line%E2%80%93plane_intersection
    
    
    Inputs
    -------
    loads: characteristic load cartisian coordinates (array) 3D space
    u_pe: envelope points in 3D space
    
    Outputs
    -------
    I: array of intersection points 
    
    Dependency
    ----------
    NA
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 07/03/2018
    
    '''
    
    hull = ConvexHull(u_pe)
    I = []
    I_temp = []
    for l_ind in np.arange(loads.shape[0]):
        
        for ind in hull.simplices:

            #Line segment 2 points
            la = np.array([[0],[0],[0]])
            lb = loads[[l_ind]].T


            #plane by 3 points
            p0 = u_pe[[ind[0]],:].T
            p1 = u_pe[[ind[1]],:].T
            p2 = u_pe[[ind[2]],:].T

            #assignments
            lab = lb - la
            p01 = p1 - p0
            p02 = p2 - p0

            temp_1 = np.hstack([-lab,p01,p02])
            temp_2 = la - p0

            para = np.dot(np.linalg.pinv(temp_1),temp_2) # parametric values [t,u,v]
            
            # if the determinant is zero then there is no unique solution i.e line is paralle or on the plane
            check_intersection_0 = np.linalg.det(temp_1)

            #If the value u+v<1, the point of intersection lies within the triangle formed by p0,p1,p2
            check_intersection_1 =  para[1]+para[2]
            
            #conditons
            # t >0: inersection is ahead of the ray 
            # u,v belongs to [0,1)
            # (u+v)<= 1 then the intersection lies within the triangle
            
            
#             print('Para = ', para)
#             print('Det = ', check_intersection_0)
#             print('sum = ', check_intersection_1)
#             print('\n')
            if (check_intersection_0 != 0) and (check_intersection_1 <= 1) and (para[:]>=0).all() and (para[1:]<=1).all():
                # intersection point
                intersection_p = la+lab*para[0]
                I_temp.append(intersection_p.T)
                
#         I_temp = np.asarray(I_temp)
#         I.append(I_temp[0][0])
#         Tracer()()
#         %matplotlib inline
#         from mpl_toolkits.mplot3d import axes3d;
#         from mpl_toolkits.mplot3d import Axes3D;
#         ax4 = plt.axes(projection='3d');
#         ax4.plot_trisurf(u_pe_newType[:,0],u_pe_newType[:,1],u_pe_newType[:,2],triangles=hull.simplices, edgecolor='k', color=(0.1, 0.2, 0.5, 0.4));
#         ax4.scatter(I_temp.reshape(2,3)[0,0],I_temp.reshape(2,3)[0,1],I_temp.reshape(2,3)[0,2], marker = 'x');
#         ax4.scatter(I_temp.reshape(2,3)[1,0],I_temp.reshape(2,3)[1,1],I_temp.reshape(2,3)[1,2], marker = 'o');

#         plt.show();
        
#         Tracer()()
        I.append(intersection_p)
    #Reshape the array
    I = np.asarray(I)
    I = I.reshape([I.shape[0],3])
    return I

In [ ]:
def build_3D_envelope_compoundBar_thermalLoads(maxPoints, r_ini, RFtol, Lk, flag_thermalAnalysisType, prop):
    
    '''
    Buils 3D envelope for a given set of points. Needs its own optimiser sovler (different probelm sunch as panels 
    or compound bar)
    
    Dependency
    ----------
    this functionc calls calculate_r_for_angles
    
    
    Inputs
    -------
    maxPoints: Lower limit of points that needs to be calculated
    r_ini: Initial guess of the actual radius
    RFtol: RF envelope tolorence for adaptive mesh refinement
    r_test_limit: radius of the unit sphere
    
    Outputs
    -------
    u_pe: critical points on the performance envelope
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 06/03/2018    
    
    '''
    u_pe =[]
    mid_ang =[0]
    while (len(u_pe) <maxPoints) and len(mid_ang) != 0 :
#         Tracer()()

        if len(u_pe) == 0:

            #set initial angles 
            ang = np.deg2rad( np.array([[0,90],[90,90],[180,90],[270,90],[0,0],[0,180]]))
            #compute points on the envelope and indices of the triangles
#             Tracer()()
            u_pe = calculate_r_for_angles_compoundBar_thermalLoads(ang, r_ini, Lk, flag_thermalAnalysisType, prop)
            tri_ind = ConvexHull(u_pe).simplices
            u_pe = np.asarray(u_pe)

        else:      

            #for each triangle, find the centroid 
            mid_ang = []
            u_pe_mid = []
            for i in np.arange(tri_ind.shape[0]):
                
                centroid = u_pe[tri_ind[i]].sum(axis = 0)/3
                
                #Adaptive mesh refinement
                
                #Check for the value at the centroid
                RF_cen,RF_ind = check_r_compoundBar_thermalLoads(centroid, Lk, flag_thermalAnalysisType, prop)
                
#                 Tracer()()
                if (RF_cen>(1+RFtol))| (RF_cen<(1-RFtol)):
                    mid_r = np.linalg.norm(centroid)
                    mid_t = np.arctan2(centroid[1],centroid[0])
                    mid_p = np.arccos(centroid[2]/mid_r)
                    mid_ang.append([mid_t,mid_p])       


            u_pe_mid = calculate_r_for_angles_compoundBar_thermalLoads(mid_ang, r_ini, Lk, flag_thermalAnalysisType, prop)
            
            u_pe_mid = np.asarray(u_pe_mid)
            u_pe = np.vstack([u_pe,u_pe_mid])


            #recompute the hull for all new points added
            tri_ind = ConvexHull(u_pe).simplices
    return u_pe

In [ ]:
def calculate_r_for_angles_compoundBar_thermalLoads(ang, r_ini, Lk, flag_thermalAnalysisType, prop):
    
    '''
    Calculates critical radius for a search direction or pair of angles passed.
    Needs optimiser solver 
    
    Inputs
    -------
    ang: angles (theat and phi) can be passed as list or numpy array
    r_ini: Initial guess of the actual radius
    r_test_limit: radius of the unit sphere
    
    Outputs
    -------
    u_pe: critical points on the performance envelope
    
    Dependency
    ----------
    this functionc calls solve_opt_test_r or similar sovler
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 06/03/2018
    
    '''
    u_pe = []
    for t,p in ang: 

        #precomputations
        ct = np.cos(t)
        sp = np.sin(p)
        st = np.sin(t)
        cp = np.cos(p)

        #Coordinate system taken from http://mathworld.wolfram.com/SphericalCoordinates.html
        #Initial points
#         char_iniX = r_ini*ct*sp
#         char_iniY = r_ini*st*sp
#         char_iniZ = r_ini*cp
        char_iniX = ct*sp
        char_iniY = st*sp
        char_iniZ = cp
        
        

        #Transform from characteristic space to actual load space
        N = np.dot(np.array([char_iniX,char_iniY,char_iniZ],ndmin =2),Lk)



        #Evaluate actual load case by optimisation and Compute critical radius
        r = solve_opt_r_compoundBar_thermalLoads(t,p, Lk, r_ini, flag_thermalAnalysisType, prop)

        #Envelope points 
        char_x = r*ct*sp
        char_y = r*st*sp
        char_z = r*cp

        u_pe.append([char_x,char_y,char_z])
        
    #Convert to numpy array format
    u_pe = np.asarray(u_pe)
    
    #Reshape the matrix to proper form
    u_pe = u_pe.reshape([u_pe.shape[0],3])
        
    return  u_pe

In [ ]:
def solve_opt_r_compoundBar_thermalLoads(t,p, Lk, r_ini, flag_thermalAnalysisType, prop):
    
    '''
    Retuns a unit value of r for testing purpose
    
    Inputs
    -------
    r_test_limit: radius of the sphere
    t: theta 
    p: phi
    
    Outputs
    -------
    res.x: Optimised value of the radius r
    
    Dependency
    ----------
    NA
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 07/03/2018
    '''

    objfun= lambda r: 1/abs(r[0])    

    def func_deriv(r):
        """ Derivative of objective function """
        dfdx0 = -1/r**2
        return np.array([ dfdx0 ])
    
    #precomputations
    ct = np.cos(t)
    sp = np.sin(p)
    st = np.sin(t)
    cp = np.cos(p)
    
    #characteristic magnitude without r
    char_x = ct*sp
    char_y = st*sp
    char_z = cp
    
    #Some discussion in PhD Notebook 8 06/03/2018. the vom mises stress is a linear function of the loads
    #s_vm = r * f(u1,u2,u3) Thus von mises can be computed with out r and then multiplied with it later on 
    il = np.dot(np.array([char_x,char_y,char_z],ndmin =2),Lk)
    il = il[0]
    
    if flag_thermalAnalysisType == 1:
        #stress in the assembly
        stress_1 = il[0]/prop['A1']+il[1]*prop['y1']/prop['I1']
        stress_1_m =  il[0]/prop['A1']-il[1]*prop['y1']/prop['I1']
        stress_2 = il[2]/prop['A2']+il[3]*prop['y2']/prop['I2']
        stress_2_m = il[2]/prop['A2']-il[3]*prop['y2']/prop['I2']
        
        cons = ({'type': 'ineq', 'fun': lambda r: -stress_1*r+ten1},
            {'type': 'ineq', 'fun': lambda r: -stress_1_m*r+ten1},
            {'type': 'ineq', 'fun': lambda r: stress_1*r-com1},
            {'type': 'ineq', 'fun': lambda r: stress_1_m*r-com1},
            {'type': 'ineq', 'fun': lambda r: -stress_2*r+ten2},
            {'type': 'ineq', 'fun': lambda r: -stress_2_m*r+ten2},
            {'type': 'ineq', 'fun': lambda r: stress_2*r-com2},
            {'type': 'ineq', 'fun': lambda r: stress_2_m*r-com2},
            )

        res = minimize(objfun, r_ini, method='SLSQP',jac=func_deriv,
        constraints=cons, options={'disp': 0})
        
    elif flag_thermalAnalysisType == 2:   
        
        #stress in the assembly
        stress_1 = (il[0]+il[2])/prop['A1']+il[1]*prop['y1']/prop['I1']
        stress_1_m =  (il[0]+il[2])/prop['A1']-il[1]*prop['y1']/prop['I1']
        stress_2 = (il[3]+il[5])/prop['A2']+il[4]*prop['y2']/prop['I2']
        stress_2_m = (il[3]+il[5])/prop['A2']-il[4]*prop['y2']/prop['I2']
            
        delT = il[2]/prop['cst_D']

        
        cons = ({'type': 'ineq', 'fun': lambda r: -stress_1*r+prop['ten1']},
                {'type': 'ineq', 'fun': lambda r: -stress_1_m*r+prop['ten1']},
                {'type': 'ineq', 'fun': lambda r: stress_1*r-prop['com1']},
                {'type': 'ineq', 'fun': lambda r: stress_1_m*r-prop['com1']},
                {'type': 'ineq', 'fun': lambda r: -stress_2*r+prop['ten2']},
                {'type': 'ineq', 'fun': lambda r: -stress_2_m*r+prop['ten2']},
                {'type': 'ineq', 'fun': lambda r: stress_2*r-prop['com2']},
                {'type': 'ineq', 'fun': lambda r: stress_2_m*r-prop['com2']},                            
                {'type': 'ineq', 'fun': lambda r: -delT*r+prop['Tmax']},
                {'type': 'ineq', 'fun': lambda r: delT*r-prop['Tmin']},
                )

        res = minimize(objfun, r_ini, method='SLSQP',jac=func_deriv,
        constraints=cons, options={'disp': 0, 'maxiter': 500})
        
    return res.x

In [ ]:
def check_r_compoundBar_thermalLoads(centroid, Lk, flag_thermalAnalysisType, prop):
    '''
    Retuns a RF value of r for testing purpose
    
    Inputs
    -------
    centroid: centroid point for a new mid point
    r_test_limit: radius of the sphere
    
    Outputs
    -------
    RF: reserve factor value 
    
    Dependency
    ----------
    NA
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 06/03/2018
    
    '''
    il = np.dot(centroid, Lk)
    
    if flag_thermalAnalysisType == 1:
        #stress in the assembly
        stress_1 = il[0]/prop['A1']+il[1]*prop['y1']/prop['I1']
        stress_1_m =  il[0]/prop['A1']-il[1]*prop['y1']/prop['I1']
        stress_2 = il[2]/prop['A2']+il[3]*prop['y2']/prop['I2']
        stress_2_m = il[2]/prop['A2']-il[3]*prop['y2']/prop['I2']

        #Check RF and return the RF index
        RF1 = ten1/(stress_1)
        RF2 = com1/(stress_1)
        RF3 = ten2/(stress_2)
        RF4 = com2/(stress_2)
        RF5 = ten1/(stress_1_m)
        RF6 = com1/(stress_1_m)
        RF7 = ten2/(stress_2_m)
        RF8 = com2/(stress_2_m)

        RF = np.array([RF1, RF2, RF3, RF4, RF5, RF6, RF7, RF8])
        
    elif flag_thermalAnalysisType == 2:    
        #stress in the assembly
        stress_1 = (il[0]+il[2])/prop['A1']+il[1]*prop['y1']/prop['I1']
        stress_1_m =  (il[0]+il[2])/prop['A1']-il[1]*prop['y1']/prop['I1']
        stress_2 = (il[3]+il[5])/prop['A2']+il[4]*prop['y2']/prop['I2']
        stress_2_m = (il[3]+il[5])/prop['A2']-il[4]*prop['y2']/prop['I2']
    
        thermal_force_tube = il[2]
        thermal_force_bar = il[5]
        delT = il[2]/prop['cst_D']
        
        #Check RF and return the RF index
        RF1 = ten1/(stress_1)
        RF2 = com1/(stress_1)
        RF3 = ten2/(stress_2)
        RF4 = com2/(stress_2)
        RF5 = ten1/(stress_1_m)
        RF6 = com1/(stress_1_m)
        RF7 = ten2/(stress_2_m)
        RF8 = com2/(stress_2_m)
        RF9 = Tmax/delT
        RF10 = Tmin/delT

        RF = np.array([RF1, RF2, RF3, RF4, RF5, RF6, RF7, RF8, RF9, RF10])
        
    RFmin = min(j for j in RF if j > 0)
    RF_ind = (np.abs(RF-RFmin)).argmin()
    
    return RFmin, RF_ind

In [3]:
def loadGen(m,v_ini,t_end,t_int, d_ratio, k):
    '''
    Generates Force from a spring mass damper system
    
    inputs = [mass,initial velocity ,time , time interval, damping ratio, spring stiffness ]
    outputs = [f, t]
    '''
    
    def MassSpringDamper(state,t,k,m,c):
        
        # unpack the state vector
        x,xd = state # displacement,x and velocity x'
        g = 9.8 # metres per second**2
        # compute acceleration xdd = x''

        xdd = -k*x/m -c*xd/m
        return [xd, xdd]
    
    t = np.arange(0,t_end,t_int)
    c_critical = 2*(k*m)**0.5 # c_critical critical damping
    c = c_critical*d_ratio
    
    #solve differential equations
    state0 = [0.0, v_ini]  #initial conditions [x0 , v0]  [m, m/sec] 
    state = odeint(MassSpringDamper, state0, t,args=(k,m,c))
    x = np.array(state[:,[0]])
    xd = np.array(state[:,[1]])
    f = x*k   
    return [f, t]


In [3]:
def sph2cart(r,t,p):
    
    '''
    Conversts spherical coordinates to carrisian coordinates in 3D
    
    Inputs
    -------
    r (array): radius
    t (arra): theta
    p (array): phi
    
    Outputs
    -------
    x (array): x coordinates
    y (array): y coordinates
    z (array): z coordinate
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 07/03/2018    
    '''
    
    #pre allocation
    ct = np.cos(t)
    sp = np.sin(p)
    st = np.sin(t)
    cp = np.cos(p)
    
    #convert to cartasian coordinate
    x = r*ct*sp
    y = r*st*sp
    z = r*cp       
    return x,y,z

def cart2sph(x,y,z):

    '''
    Conversts cartisian coordinates to spherical coordinates in 3D
    
    Inputs
    -------
    x (array): x coordinates
    y (array): y coordinates
    z (array): z coordinate
    
    Outputs
    -------
    r (array): radius
    t (arra): theta
    p (array): phi
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 07/03/2018    
    '''
    
    r = np.linalg.norm([x,y,z], axis = 0)
    t = np.arctan2(y,x)
    p = np.arccos(z/r) 
    return r,t,p


def cart2pol(x, y):
    
    '''
    Conversts cartisian coordinates to polar coordinates in 2D
    
    Inputs
    -------
    x (array): x coordinates
    y (array): y coordinates

    
    Outputs
    -------
    r (array): radius
    t (arra): theta

    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: ----  
    ''' 

    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    phi = ( phi + 2*np.pi) % (2 * np.pi )# - np.pi  #Wraps angle from [0,pi] to [0,2pi]
    
    return(phi,rho)

def pol2cart(phi,rho):
    
    '''
    Conversts polar coordinates to cartisian coordinates in 2D
    
    Inputs
    -------
    r (array): radius
    t (arra): theta
    
    Outputs
    -------
    x (array): x coordinates
    y (array): y coordinates
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: ----  
    ''' 
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

In [5]:
def find_intersection(x2,x3,x4):
    
    '''
    
    finds the intersection between two vectors made up of points x1, x2 and x3, x4. x1 is origin thus (0,0)
    
    '''
 
    A = np.array([[x2[0],x3[0]-x4[0]],[x2[1],x3[1]-x4[1]]])
    B = np.array ([x3[0],x3[1]])
    X = np.dot(np.linalg.pinv(A),B)

#     X = np.linalg.solve(A, B)
    Ix,Iy = X[0]*x2[0], X[0]*x2[1]
    
    #Direct way
#     Tracer()()    
#     Ix = (x2[0]*(x3[0]*x4[1]-x3[1]*x4[0]))/(-x3[0]*(x3[1]-x4[1])+x2[1]*(x3[0]-x4[0]))
#     Iy = (x2[1]*(x3[0]*x4[1]-x3[1]*x4[0]))/(-x2[0]*(x3[1]-x4[1])+x2[1]*(x3[0]-x4[0]))
    return (Ix,Iy)

In [4]:
def buildEnvelope_compoundBar_2D(prop, r_ini, lk, theta): #,flag_refine,maxPoints):
    
    
    """
    Example problem 

    def testOpt():
        objfun= lambda r: 1/abs(r[0])
        ct=-1

        h = -ct

        cons = ({'type': 'ineq', 'fun': lambda r: h*r+600},
                {'type': 'ineq', 'fun': lambda r: ct*r+200},
                {'type': 'ineq', 'fun': lambda r: h*r+400},
               )
        res = minimize(objfun, (1), method='SLSQP', tol=1e-6, constraints=cons, options={'disp': True})
        return res.x

    r = testOpt()
    print(r)
    """
    A1 = prop['A1']
    A2 = prop['A2']
    y1 = prop['y1']
    y2 = prop['y2']
    I1 = prop['I1']
    I2 = prop['I2']
    ten1 = prop['ten1']
    ten2 = prop['ten2']
    com1 = prop['com1']
    com2 = prop['com2']
    

    cnt = 0
    charLoad = np.zeros([theta.shape[0],2])
    RF = np.zeros([theta.shape[0],8])
    RF_ind = np.zeros([theta.shape[0],1])
    
    for i in theta:
        
        objfun= lambda r: 1/abs(r[0])    
        
        def func_deriv(r):
            """ Derivative of objective function """
            dfdx0 = -1/r**2
            return np.array([ dfdx0 ])
        
        ct=np.cos(i)
        st=np.sin(i)
        
        #Internal load matrix example is il=[f1,m1,f2,m2]
        il=np.dot(lk.T,np.array([[ct],[st]]))
                  
        #stress in the assembly
        stress_1 = il[0]/A1+il[1]*y1/I1
        stress_1_m =  il[0]/A1-il[1]*y1/I1
        stress_2 = il[2]/A2+il[3]*y2/I2
        stress_2_m = il[2]/A2-il[3]*y2/I2
       

        cons = ({'type': 'ineq', 'fun': lambda r: -stress_1*r+prop['ten1']},
                {'type': 'ineq', 'fun': lambda r: -stress_1_m*r+prop['ten1']},
                {'type': 'ineq', 'fun': lambda r: stress_1*r-prop['com1']},
                {'type': 'ineq', 'fun': lambda r: stress_1_m*r-prop['com1']},
                {'type': 'ineq', 'fun': lambda r: -stress_2*r+prop['ten2']},
                {'type': 'ineq', 'fun': lambda r: -stress_2_m*r+prop['ten2']},
                {'type': 'ineq', 'fun': lambda r: stress_2*r-prop['com2']},
                {'type': 'ineq', 'fun': lambda r: stress_2_m*r-prop['com2']},
                )
#         Tracer()()
        res = minimize(objfun, (0.01), method='SLSQP',jac=func_deriv,
        constraints=cons, options={'disp': 0})
        
        #Check RF and return the RF index
        RF1 = ten1/(stress_1*res.x)
        RF2 = com1/(stress_1*res.x)
        RF3 = ten2/(stress_2*res.x)
        RF4 = com2/(stress_2*res.x)
        RF5 = ten1/(stress_1_m*res.x)
        RF6 = com1/(stress_1_m*res.x)
        RF7 = ten2/(stress_2_m*res.x)
        RF8 = com2/(stress_2_m*res.x)
        
        RF[cnt,:] = RF1,RF2,RF3,RF4,RF5,RF6,RF7,RF8
        RF_ind[cnt,:] = (np.abs(RF[cnt,:]-1)).argmin()
                    
        charLoad[cnt,:] = res.x*ct,res.x*st  
        cnt = cnt+1            
        
    return charLoad, RF_ind

In [7]:
def refineEnvelope(prop, u_pe, maxPoints, Il_lk):
    
    '''
    
    '''
    cnt = 0
    thetaMid_recal = np.array([0])
    
    while (len(u_pe) <maxPoints) and thetaMid_recal.size != 0 :
        cnt = cnt +1
        
        #Generate mid points on the initial envelope 
        xMid = np.zeros([len(u_pe)-1, 1])
        yMid = np.zeros([len(u_pe)-1, 1])

        for i in range(0,len(u_pe)-1):
            xMid[i,0] = (u_pe[i,0]+u_pe[i+1,0])/2
            yMid[i,0] = (u_pe[i,1]+u_pe[i+1,1])/2

        #Convert to polar coordinates
        thetaMid,rMid = cart2pol(xMid,yMid)

        #Evaluate RF for the mid points
        RFmin, RF_ind = checkRF_2D(prop, Il_lk, rMid, thetaMid)

        #Check if the RFmin is within tol range
        ind_RF_greater = RFmin >=(1+RFtol)
        ind_RF_lesser = RFmin <=(1-RFtol)

        #Compute the index of the theta which needs recalculation
        ind_RF_recal = ind_RF_greater|ind_RF_lesser 

        thetaMid_critical = thetaMid[ind_RF_recal]
        r_mid_critical =rMid[ind_RF_recal]

        #Compute critical radius for the new mid points
#         Tracer()()
        u_pe_mid, RF_ind_mid = buildEnvelope_compoundBar_2D(prop, r_ini, Il_lk, thetaMid_critical)#,False,0) #Make sure flag_refine is 0
#         #remove the last row added earlier
#         u_pe_mid = np.delete(u_pe_mid, (-1), axis=0)
        
        #Add in the critical radius to the mid points

        thetaMid_recal,rMid_recal = cart2pol(u_pe_mid[:,[0]],u_pe_mid[:,[1]])
        rMid[ind_RF_recal] = np.squeeze(rMid_recal)
        
        #Convert theta and r for the points on the envelope
        theta_pe,r_pe = cart2pol(u_pe[:,[0]],u_pe[:,[1]])

        u_pe_pol = np.hstack([theta_pe,r_pe])

        #remove the last row added earlier
        u_pe_pol = np.delete(u_pe_pol, (-1), axis=0)

        #add new points to the list
#         u_pe_pol = np.vstack([u_pe_pol,np.hstack([thetaMid,rMid])])
        u_pe_pol = np.vstack([u_pe_pol,np.hstack([thetaMid_recal,rMid_recal])])

        #Sort out by theta
        u_pe_pol = u_pe_pol[u_pe_pol[:,0].argsort(),]

        #convert back to cart
        xPE,yPE = pol2cart(u_pe_pol[:,[0]],u_pe_pol[:,[1]])

        #update final u_pe
        u_pe = np.hstack([xPE,yPE])

        #Add in last row equal to the first so the plot comes out good
        u_pe = np.vstack([u_pe,u_pe[0,:]])
        
    return u_pe

In [19]:
def envelope_2D_BuildandRefine(prop, r_ini, lk, thetaInitial, flag_refine, maxPoints):
    
    '''
    
    '''
    
    u_pe1, RF_ind_ini = buildEnvelope_compoundBar_2D(prop, r_ini, lk, thetaInitial)
    u_pe1 = np.vstack([u_pe1,u_pe1[0,:]]) #connects the last one line
    
    if flag_refine == 1:
        u_pe = refineEnvelope(prop, u_pe1, maxPoints, lk)
        
    else:
        u_pe = u_pe1  
    
    return u_pe
    

In [8]:
def checkRF_2D(prop, lk, r_mid, theta_mid):
    
    '''
    
    '''
    A1 = prop['A1']
    A2 = prop['A2']
    y1 = prop['y1']
    y2 = prop['y2']
    I1 = prop['I1']
    I2 = prop['I2']
    ten1 = prop['ten1']
    ten2 = prop['ten2']
    com1 = prop['com1']
    com2 = prop['com2']
    
    cnt = 0
    RF = np.zeros([theta_mid.shape[0],8])
    RF_ind = np.zeros([theta_mid.shape[0],1])
    RFmin = np.zeros([theta_mid.shape[0],1])
    
    for i in theta_mid:
        
        ct=np.cos(i)
        st=np.sin(i)
        ctst = np.zeros([2,1])
        ctst = np.array([ct,st])
        
        #Internal load matrix example is il=[f1,m1,f2,m2]
        il=np.dot(lk.T,ctst)
                  
        #stress in the assembly
        stress_1 = il[0]/A1+il[1]*y1/I1
        stress_1_m =  il[0]/A1-il[1]*y1/I1
        stress_2 = il[2]/A2+il[3]*y2/I2
        stress_2_m = il[2]/A2-il[3]*y2/I2
        
        #Check RF and return the RF index
        RF1 = ten1/(stress_1*r_mid[cnt])
        RF2 = com1/(stress_1*r_mid[cnt])
        RF3 = ten2/(stress_2*r_mid[cnt])
        RF4 = com2/(stress_2*r_mid[cnt])
        RF5 = ten1/(stress_1_m*r_mid[cnt])
        RF6 = com1/(stress_1_m*r_mid[cnt])
        RF7 = ten2/(stress_2_m*r_mid[cnt])
        RF8 = com2/(stress_2_m*r_mid[cnt])
        
        RF[cnt,:] = RF1,RF2,RF3,RF4,RF5,RF6,RF7,RF8
        
        RFmin[cnt,:] = min(j for j in RF[cnt,:] if j > 0)
        
        RF_ind[cnt,:] = (np.abs(RF[cnt,:]-RFmin[cnt,:])).argmin()

        cnt = cnt+1            

    return RFmin, RF_ind


In [9]:
def FastRF_2D(u_pe, u_lc):
    
    '''
    
    Inputs in cartisian coordinate system
    
    '''
    
    #convert envelope points to polar coordinates
    u_pe_pol = np.zeros([len(u_pe),2])
    u_pe_pol[:,[0]],u_pe_pol[:,[1]] = cart2pol(u_pe[:,[0]], u_pe[:,[1]])
    
    #remove the last row added earlier
    u_pe_pol = np.delete(u_pe_pol, (-1), axis=0)
    
    #Convert load cases to polar coordinates

    u_lc_pol = np.zeros([len(u_lc),2])
    u_lc_pol[:,[0]],u_lc_pol[:,[1]] = cart2pol(u_lc[:,[0]], u_lc[:,[1]])
    
    cnt = 0
    NN = np.zeros([len(u_lc),2])
    
    #Allowable load case
    allowable_lc = np.zeros([len(u_lc),2])
    
    #RF
    RF_est = np.zeros([len(u_lc),1])
    
    #Find the facet intersecting the load case
    for theta, actual_rad_lc in u_lc_pol:
#         Tracer()()
       
        idx_grt = (u_pe_pol[:,[0]]>=theta)
        idx_lss = (u_pe_pol[:,[0]]<=theta)


        NN[cnt,[0]] = np.where(idx_lss)[0][-1]
        
        if not np.where(idx_grt)[0].size:
            NN[cnt,[1]] = np.array([0])
        else:
            NN[cnt,[1]] = np.where(idx_grt)[0][0]       
    
        #Find the intersection between two vectors
        x3 = np.array([u_pe[int(NN[cnt,0]),0],u_pe[int(NN[cnt,0]),1]])
        x4 = np.array([u_pe[int(NN[cnt,1]),0],u_pe[int(NN[cnt,1]),1]])
        x2 = u_lc[cnt,:]
        allowable_lc[cnt,0],allowable_lc[cnt,1] = find_intersection(x2,x3,x4)

        the_temp, allowable_rad = cart2pol(allowable_lc[cnt,0], allowable_lc[cnt,1]) 
        
        RF_est[cnt] = allowable_rad/actual_rad_lc
        cnt = cnt+1
        
    return RF_est

In [10]:
def sensitivityFromEnvelope (u_pe,C_ind,perFac):
    
    '''
    
    '''
    dRFdU_PE = np.zeros([8,2])
    u_pe_normal_points = np.zeros([6,2])
    pmid_points = np.zeros([6,2])
    p1_points = np.zeros([6,2])
    p2_points = np.zeros([6,2])

    for ind, c_ind in enumerate(C_ind):
        p1 = u_pe[c_ind,:]
        p2 = u_pe[c_ind+1,:]
        pmid = (p1+p2)/2
        p1_points[ind,0], p1_points[ind,1] = p1[0], p1[1]
        p2_points[ind,0], p2_points[ind,1] = p2[0], p2[1]
        pmid_points[ind,0], pmid_points[ind,1] = pmid[0], pmid[1]


        facetVector = np.array([p1[0]-p2[0],p1[1]-p2[1]])
        facetVector_normal = np.array([facetVector[1],-facetVector[0]])

        normal_dri = np.arctan2(facetVector_normal[1],facetVector_normal[0])
        normal_mag = np.linalg.norm(facetVector_normal)

        #New point normal to facet
        del_r_normal = normal_mag*perFac
        del_x_normal = del_r_normal*np.cos(normal_dri)
        del_y_normal = del_r_normal*np.sin(normal_dri)

    #     u_pe_normal = np.array([u_pe[c_ind,0]+del_x_normal,u_pe[c_ind,1]+del_y_normal])
        u_pe_normal = np.array([pmid[0]+del_x_normal, pmid[1]+del_y_normal])
        u_pe_normal_points[ind,0], u_pe_normal_points[ind,1] = u_pe_normal[0], u_pe_normal[1]

        normalVector =  np.array([pmid[0]-u_pe_normal[0],pmid[1]-u_pe_normal[1]])

    #     print(np.dot(facetVector,normalVector))

        #Compute RF from envelope
        temp_U1 = np.zeros([1,2])
        temp_U2 = np.zeros([1,2])
        temp_U1[0,0], temp_U1[0,1] = u_pe_normal[[0]], pmid[[1]]
        temp_U2[0,0], temp_U2[0,1] = pmid[0], u_pe_normal[1]

        #dU1
        RF_pe_U1 =FastRF_2D(u_pe,temp_U1)
        #dU2
        RF_pe_U2 =FastRF_2D(u_pe,temp_U2)

#         theta_loads,r_loads = cart2pol(u_pe_normal[[0]],pmid[[1]])
#         RFmin_testLoads, RF_ind_loads = checkRF_2D(E1, A1, I1, y1, ten1, com1, E2, A2, I2, y2, ten2, com2, Il_lk, r_loads, theta_loads)

        #dRF/dU1
        dRF_dU1 = (1-RF_pe_U1)/del_x_normal


        #dRF/dU2
        dRF_dU2 = (1-RF_pe_U2)/del_y_normal

        dRFdU_PE[int(C[ind]),0], dRFdU_PE[int(C[ind]),1] = dRF_dU1, dRF_dU2
        
    return dRFdU_PE, u_pe_normal_points, p1_points, p2_points, pmid_points

In [11]:
def sensitivityFromAnalyticalSolution(u_pe, C, prop, Il_lk):
    '''
    
    From solving analytical equations Notebook 7 10/10/2017
    RF4=com2/((eN1+fN2/A2)+(gN1+hN2)y2/I2) where e,f,g,h are the values in lk
    corresponding to the internal load cl3 and cl4
    
    '''
    A1 = prop['A1']
    A2 = prop['A2']
    y1 = prop['y1']
    y2 = prop['y2']
    I1 = prop['I1']
    I2 = prop['I2']
    ten1 = prop['ten1']
    ten2 = prop['ten2']
    com1 = prop['com1']
    com2 = prop['com2']
    
    tempLk = Il_lk.T
    a = tempLk[0,0]
    b = tempLk[0,1]
    c = tempLk[1,0]
    d = tempLk[1,1]
    e = tempLk[2,0]
    f = tempLk[2,1]
    g = tempLk[3,0]
    h = tempLk[3,1]
    
    # Analytical solutions
    dRFdU_ana = np.zeros([8,2])
    for ind, c_cir in enumerate(C):

        #RF1
        if c_cir == 0:
            N1_RF1 , N2_RF1  = u_pe[C_ind[ind],:]
            dRF1dN1_ana = (ten1*(a/A1 + (c*y1)/I1))/((N1_RF1*a + N2_RF1*b)/A1 + (y1*(N1_RF1*c + N2_RF1*d))/I1)**2
            dRF1dN2_ana = (ten1*(b/A1 + (d*y1)/I1))/((N1_RF1*a + N2_RF1*b)/A1 + (y1*(N1_RF1*c + N2_RF1*d))/I1)**2
            dRFdU_ana[0,0],dRFdU_ana[0,1] = dRF1dN1_ana,dRF1dN2_ana

        #RF2
        if c_cir == 1:
            N1_RF2 , N2_RF2  = u_pe[C_ind[ind],:]
            dRF2dN1_ana = -(com1*(a/A1 + (c*y1)/I1))/((N1_RF2*a + N2_RF2*b)/A1 + (y1*(N1_RF2*c + N2_RF2*d))/I1)**2
            dRF2dN2_ana = -(com1*(b/A1 + (d*y1)/I1))/((N1_RF2*a + N2_RF2*b)/A1 + (y1*(N1_RF2*c + N2_RF2*d))/I1)**2
            dRFdU_ana[1,0],dRFdU_ana[1,1] = dRF2dN1_ana,dRF2dN2_ana

        #RF3
        if c_cir == 2:
            N1_RF3 , N2_RF3  = u_pe[C_ind[ind],:]
            dRF3dN1_ana = (ten2*(e/A2 + (g*y2)/I2))/((N1_RF3*e + N2_RF3*f)/A2 + (y2*(N1_RF3*g + N2_RF3*h))/I2)**2
            dRF3dN2_ana = (ten2*(f/A2 + (h*y2)/I2))/((N1_RF3*e + N2_RF3*f)/A2 + (y2*(N1_RF3*g + N2_RF3*h))/I2)**2
            dRFdU_ana[2,0],dRFdU_ana[2,1] = dRF3dN1_ana,dRF3dN2_ana

        #RF4
        if c_cir == 3:
            N1_RF4 , N2_RF4  = u_pe[C_ind[ind],:]
            dRF4dN1_ana = (com2*(e/A2 + (g*y2)/I2))/((N1_RF4*e + N2_RF4*f)/A2 + (y2*(N1_RF4*g + N2_RF4*h))/I2)**2
            dRF4dN2_ana = (com2*(f/A2 + (h*y2)/I2))/((N1_RF4*e + N2_RF4*f)/A2 + (y2*(N1_RF4*g + N2_RF4*h))/I2)**2
            dRFdU_ana[3,0],dRFdU_ana[3,1] = dRF4dN1_ana,dRF4dN2_ana

        #RF5
        if c_cir == 4:
            N1_RF5 , N2_RF5  = u_pe[C_ind[ind],:]
            dRF5dN1_ana = (ten1*(a/A1 - (c*y1)/I1))/((N1_RF5*a + N2_RF5*b)/A1 - (y1*(N1_RF5*c + N2_RF5*d))/I1)**2
            dRF5dN2_ana = (ten1*(b/A1 - (d*y1)/I1))/((N1_RF5*a + N2_RF5*b)/A1 - (y1*(N1_RF5*c + N2_RF5*d))/I1)**2
            dRFdU_ana[4,0],dRFdU_ana[4,1] = dRF5dN1_ana,dRF5dN2_ana

        #RF6
        if c_cir == 5:
            N1_RF6 , N2_RF6  = u_pe[C_ind[ind],:]
            dRF6dN1_ana = -(com1*(a/A1 - (c*y1)/I1))/((N1_RF6*a + N2_RF6*b)/A1 - (y1*(N1_RF6*c + N2_RF6*d))/I1)**2
            dRF6dN2_ana = -(com1*(b/A1 - (d*y1)/I1))/((N1_RF6*a + N2_RF6*b)/A1 - (y1*(N1_RF6*c + N2_RF6*d))/I1)**2
            dRFdU_ana[5,0],dRFdU_ana[5,1] = dRF6dN1_ana,dRF6dN2_ana

        #RF7
        if c_cir == 6:
            N1_RF7 , N2_RF7  = u_pe[C_ind[ind],:]
            dRF7dN1_ana = (ten2*(e/A2 - (g*y2)/I2))/((N1_RF7*e + N2_RF7*f)/A2 - (y2*(N1_RF7*g + N2_RF7*h))/I2)**2
            dRF7dN2_ana = (ten2*(f/A2 - (h*y2)/I2))/((N1_RF7*e + N2_RF7*f)/A2 - (y2*(N1_RF7*g + N2_RF7*h))/I2)**2
            dRFdU_ana[6,0],dRFdU_ana[6,1] = dRF7dN1_ana,dRF7dN2_ana

        #RF8
        if c_cir == 7:
            N1_RF8 , N2_RF8  = u_pe[C_ind[ind],:]
            dRF8dN1_ana = (com2*(e/A2 - (g*y2)/I2))/((N1_RF8*e + N2_RF8*f)/A2 - (y2*(N1_RF8*g + N2_RF8*h))/I2)**2
            dRF8dN2_ana = (com2*(f/A2 - (h*y2)/I2))/((N1_RF8*e + N2_RF8*f)/A2 - (y2*(N1_RF8*g + N2_RF8*h))/I2)**2
            dRFdU_ana[7,0],dRFdU_ana[7,1] = dRF8dN1_ana,dRF8dN2_ana
    return dRFdU_ana

In [12]:
def analyticalSensitivity_dRFdN(prop, u_pe_cri, conNum, Lk1):
    
    A1 = prop['A1']
    A2 = prop['A2']
    y1 = prop['y1']
    y2 = prop['y2']
    I1 = prop['I1']
    I2 = prop['I2']
    ten1 = prop['ten1']
    ten2 = prop['ten2']
    com1 = prop['com1']
    com2 = prop['com2']
    
    f1,m1,f2,m2 = np.dot(u_pe_cri,Lk1)
    
    if conNum == 0:
        dRFdf1 = (-ten1)/(A1*(f1/A1 + (m1*y1)/I1)**2)
        dRFdm1 = -(ten1*y1)/(I1*(f1/A1 + (m1*y1)/I1)**2)
        dRFdf2 = 0
        dRFdm2 = 0
    
    if conNum == 1:
        dRFdf1 = (-com1)/(A1*(f1/A1 + (m1*y1)/I1)**2)
        dRFdm1 = -(com1*y1)/(I1*(f1/A1 + (m1*y1)/I1)**2)
        dRFdf2 = 0
        dRFdm2 = 0

    if conNum == 2:
        dRFdf2 = -ten2/(A2*(f2/A2 + (m2*y2)/I2)**2)
        dRFdm2 = -(ten2*y2)/(I2*(f2/A2 + (m2*y2)/I2)**2)
        dRFdf1 = 0
        dRFdm1 = 0

    if conNum == 3:    
        dRFdf2 = -com2/(A2*(f2/A2 + (m2*y2)/I2)**2)
        dRFdm2 = -(com2*y2)/(I2*(f2/A2 + (m2*y2)/I2)**2)
        dRFdf1 = 0
        dRFdm1 = 0
        
    if conNum == 4:
        dRFdf1 = -ten1/(A1*(f1/A1 - (m1*y1)/I1)**2)
        dRFdm1 = (ten1*y1)/(I1*(f1/A1 - (m1*y1)/I1)**2)
        dRFdf2 = 0
        dRFdm2 = 0
       
    if conNum == 5:
        dRFdf1 = -com1/(A1*(f1/A1 - (m1*y1)/I1)**2)
        dRFdm1 = (com1*y1)/(I1*(f1/A1 - (m1*y1)/I1)**2)
        dRFdf2 = 0
        dRFdm2 = 0
        
    if conNum == 6:
        dRFdf2 = -ten2/(A2*(f2/A2 - (m2*y2)/I2)**2)
        dRFdm2 = (ten2*y2)/(I2*(f2/A2 - (m2*y2)/I2)**2)
        dRFdf1 = 0
        dRFdm1 = 0

    if conNum ==7:
        dRFdf2 = -com2/(A2*(f2/A2 - (m2*y2)/I2)**2)
        dRFdm2 = (com2*y2)/(I2*(f2/A2 - (m2*y2)/I2)**2)
        dRFdf1 = 0
        dRFdm1 = 0

    dRFdN_ana = np.array([[dRFdf1,dRFdm1,dRFdf2,dRFdm2]])

    return dRFdN_ana

In [13]:
def analyticalSensitivity_dNdx(Lk1, u_pe_cri, prop):
    
    x1 = prop['x1']
    x2 = prop['x2']
    x3 = prop['x3']
    E1 = prop['E1']
    E2 = prop['E2']
    
    f1,m1,f2,m2 = np.dot(u_pe_cri,Lk1)
    F = f1+f2
    M = m1+m2
    
    #dN/dx1
    dft_dx1 =  (2*E1*F*np.pi*x1)/(np.pi*E1*(x1**2 - x2**2) + np.pi*E2*x3**2) - (2*E1**2*F*np.pi**2*x1*(x1**2 - x2**2))/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2

    dmt_dx1 = (E1*M*np.pi*x1**3)/((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4) - (E1**2*M*np.pi**2*x1**3*(x1**4 - x2**4))/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2)

    dfb_dx1 = -(2*E1*E2*F*np.pi**2*x1*x3**2)/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2

    dmb_dx1 = -(E1*E2*M*np.pi**2*x1**3*x3**4)/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2)

    dN_dx1 = np.array([[dft_dx1],[dmt_dx1],[dfb_dx1],[dmb_dx1]])
    
    #dN/dx2
    dft_dx2 = (2*E1**2*F*np.pi**2*x2*(x1**2 - x2**2))/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2 - (2*E1*F*np.pi*x2)/(np.pi*E1*(x1**2 - x2**2) + np.pi*E2*x3**2)
    
    dmt_dx2 = (E1**2*M*np.pi**2*x2**3*(x1**4 - x2**4))/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2) - (E1*M*np.pi*x2**3)/((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)
    
    dfb_dx2 = (2*E1*E2*F*np.pi**2*x2*x3**2)/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2
    
    dmb_dx2 = (E1*E2*M*np.pi**2*x2**3*x3**4)/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2)
    
    #dN/dx3

    dft_dx3 = -(2*E1*E2*F*np.pi**2*x3*(x1**2 - x2**2))/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2
    
    dmt_dx3 = -(E1*E2*M*np.pi**2*x3**3*(x1**4 - x2**4))/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2) 
    
    dfb_dx3 = (2*E2*F*np.pi*x3)/(np.pi*E1*(x1**2 - x2**2) + np.pi*E2*x3**2) - (2*E2**2*F*np.pi**2*x3**3)/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2
    
    dmb_dx3 = (E2*M*np.pi*x3**3)/((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4) - (E2**2*M*np.pi**2*x3**7)/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2)

    dN_dx1 = np.vstack((dft_dx1,dmt_dx1,dfb_dx1,dmb_dx1))
    dN_dx2 = np.vstack((dft_dx2,dmt_dx2,dfb_dx2,dmb_dx2))
    dN_dx3 = np.vstack((dft_dx3,dmt_dx3,dfb_dx3,dmb_dx3))
    
#     dN_dx = np.hstack((dN_dx1,dN_dx2,dN_dx3))
    return dN_dx1, dN_dx2, dN_dx3


In [2]:
def change_of_basis_multiCompoundBar(base_lk, Lk1, u_pe_len_count, u_pe):
    
    ''' 
    
    Changes the basis of the characteristic loads to a base characteristic load
    Inputs = [base_lk, Lk1, u_pe_len_count, u_pe]
    Outputs = [u_pe_baseTrans, T, T_sf, u_pe_sf]
    
    '''
    
    T = np.zeros([1,2])
    u_pe_baseTrans = np.zeros([1,2])
    cnt = 0
    cnt_u = 0
    for j in range(len(u_pe_len_count)):
    
        #Get number of characteristic loads per sf, usually it is 2
        lknum = int(len(Lk1)/len(u_pe_len_count))

        #Get characteristic load per sf
        lk_sf = Lk1[cnt:cnt+lknum,:]

        #Find the transformation matrix of that sf
        T_sf = np.dot(base_lk,np.linalg.pinv(lk_sf))

        #Global transformation matrix
        T = np.append(T,T_sf, axis = 0)

        #Slice the u_pe per sf
        u_pe_sf = u_pe[cnt_u:cnt_u+u_pe_len_count[j],:]

        #Transform u_pe of sf to a new basis
        u_pe_baseTrans_sf = np.dot(u_pe_sf,np.linalg.pinv(T_sf))

        #Append to the global matrix
        u_pe_baseTrans = np.append(u_pe_baseTrans,u_pe_baseTrans_sf, axis = 0)

        cnt = cnt + lknum
        cnt_u = cnt_u+u_pe_len_count[j]
        
    u_pe_baseTrans = np.delete(u_pe_baseTrans,[0],axis=0)
    T = np.delete(T,[0],axis=0)
        
    return u_pe_baseTrans, T, T_sf, u_pe_sf

In [2]:
def internal_characteristicLoads_performancEnvelopes_multiCompoundBar(cb_matProp, P, k1, r_ini, thetaInitial, 
                                                                      flag_refine, maxPoints):
    
    ''' 
    1. Precompute A1, A2, I1, I2
    2. Compute internal loads for all compound bars
    3. Identify the characteristic loads for all compound bars
    4. Build performance envelope for each compound bar
    5. Save all the individual data
    
    Inputs
    ------
    cb_matProp:
    P:
    k1:
    r_ini:
    thetaInitial:
    flag_refine:
    maxPoints:
    
    Outputs
    -------
    N:
    Lk1:
    Uk1:
    u_pe:
    u_pe_sf:
    Lk1_sf:
    u_pe_len_count:
    
    '''
    #Pre computations
    A1 =  np.pi*(cb_matProp.X1**2 - cb_matProp.X2**2)
    A2 = np.pi*cb_matProp.X3**2
    I2 = (np.pi/4)*cb_matProp.X3**4
    I1 = (np.pi/4)*(cb_matProp.X1**4 - cb_matProp.X2**4)

    sum_EA = np.sum(cb_matProp.E1 * A1 + cb_matProp.E2 * A2 )
    sum_EI = np.sum(cb_matProp.E1 * I1 + cb_matProp.E2 * I2 )
    N = np.zeros([1,4])
    Lk_N0 = np.zeros([1,4])
    Lk1 = np.zeros([1,4])
    Uk1 = np.zeros([1,2])
    u_pe_len_count = []
    u_pe = np.empty([1,2])

    for ind in list(range(cb_matProp.shape[0])):

        gamma = np.array([[(cb_matProp.E1[ind] * A1[ind])/sum_EA, 0, (cb_matProp.E2[ind] * A2[ind])/sum_EA,0]
                          ,[0,(cb_matProp.E1[ind] * I1[ind])/sum_EI, 0 ,(cb_matProp.E2[ind] * I2[ind] )/sum_EI ]])

        #Actual internal loads per compound bar/structural feature
        N_sf = np.dot(P,gamma) 

        #global internal load matrix
        N = np.append(N,N_sf, axis =0)

        #Internal characteristic loads from external characteristic loads per structural feature
        Lk_N0_sf = np.dot(Lk0,gamma) 

        #global internal characteristic matrix
        Lk_N0 = np.append(Lk_N0,Lk_N0_sf, axis = 0)

        #Characteristic internal load per structural feature
        Uk1_sf,Lk1_sf,err_sf = svd_reducedrank(Lk_N0_sf, k1,  0) # (A, k, flag_balance)

        #global internal characteristic response to external characteristic matrix
        Lk1 = np.append(Lk1,Lk1_sf, axis = 0)
        Uk1 = np.append(Uk1,Uk1_sf, axis = 0)
        
        prop = dict()
        prop['E1'] = cb_matProp.E1[ind]
        prop['A1'] = A1[ind]
        prop['I1'] = I1[ind]
        prop['y1'] = cb_matProp.X1[ind]
        prop['ten1'] = cb_matProp.ten1[ind]
        prop['com1'] = cb_matProp.com1[ind]
        prop['E2'] = cb_matProp.E2[ind]
        prop['A2'] = A2[ind]
        prop['I2'] = I2[ind]
        prop['y2'] = cb_matProp.X3[ind]
        prop['ten2'] = cb_matProp.ten2[ind]
        prop['com2'] = cb_matProp.com2[ind]

        #Performance envelopes for each structural feature    
        u_pe_sf = envelope_2D_BuildandRefine(prop, r_ini, Lk1_sf, thetaInitial, flag_refine, maxPoints) #(r_ini, lk, thetaInitial, flag_refine, maxPoints)
        #Count of u_pe_sf to be used for seperating u_pe later
        u_pe_len_count.append(len(u_pe_sf))

        #Global u_pe 
        u_pe = np.append(u_pe,u_pe_sf, axis = 0)

    #delete the first row of the matrix
    N = np.delete(N,[0],axis=0)
    Lk_N0 = np.delete(Lk_N0,[0],axis=0)
    Lk1 = np.delete(Lk1,[0],axis=0)
    Uk1 = np.delete(Uk1,[0],axis=0)
    u_pe = np.delete(u_pe,[0],axis=0)
    
    
    return N, Lk1, Uk1, u_pe, u_pe_sf, Lk1_sf, u_pe_len_count


SyntaxError: invalid syntax (<ipython-input-2-d3c5bfc3fef4>, line 71)

In [2]:
def analyticalSensitivity_dRFdN_V3(prop, N, conNum):
    
    '''
    dRF/dN for any load case given the constraint
    
    input
    ------
    
    prop:
    
    N: Internal loads fft, mt, fb, mb
    
    conNum: constraint number to choose between 0 and 7
    '''
    
    A1 = prop['A1']
    A2 = prop['A2']
    y1 = prop['y1']
    y2 = prop['y2']
    I1 = prop['I1']
    I2 = prop['I2']
    ten1 = prop['ten1']
    ten2 = prop['ten2']
    com1 = prop['com1']
    com2 = prop['com2']
    
    f1,m1,f2,m2 = N
    
    if conNum == 0:
        dRFdf1 = (-ten1)/(A1*(f1/A1 + (m1*y1)/I1)**2)
        dRFdm1 = -(ten1*y1)/(I1*(f1/A1 + (m1*y1)/I1)**2)
        dRFdf2 = 0
        dRFdm2 = 0
    
    if conNum == 1:
        dRFdf1 = (-com1)/(A1*(f1/A1 + (m1*y1)/I1)**2)
        dRFdm1 = -(com1*y1)/(I1*(f1/A1 + (m1*y1)/I1)**2)
        dRFdf2 = 0
        dRFdm2 = 0

    if conNum == 2:
        dRFdf2 = -ten2/(A2*(f2/A2 + (m2*y2)/I2)**2)
        dRFdm2 = -(ten2*y2)/(I2*(f2/A2 + (m2*y2)/I2)**2)
        dRFdf1 = 0
        dRFdm1 = 0

    if conNum == 3:    
        dRFdf2 = -com2/(A2*(f2/A2 + (m2*y2)/I2)**2)
        dRFdm2 = -(com2*y2)/(I2*(f2/A2 + (m2*y2)/I2)**2)
        dRFdf1 = 0
        dRFdm1 = 0
        
    if conNum == 4:
        dRFdf1 = -ten1/(A1*(f1/A1 - (m1*y1)/I1)**2)
        dRFdm1 = (ten1*y1)/(I1*(f1/A1 - (m1*y1)/I1)**2)
        dRFdf2 = 0
        dRFdm2 = 0
       
    if conNum == 5:
        dRFdf1 = -com1/(A1*(f1/A1 - (m1*y1)/I1)**2)
        dRFdm1 = (com1*y1)/(I1*(f1/A1 - (m1*y1)/I1)**2)
        dRFdf2 = 0
        dRFdm2 = 0
        
    if conNum == 6:
        dRFdf2 = -ten2/(A2*(f2/A2 - (m2*y2)/I2)**2)
        dRFdm2 = (ten2*y2)/(I2*(f2/A2 - (m2*y2)/I2)**2)
        dRFdf1 = 0
        dRFdm1 = 0

    if conNum ==7:
        dRFdf2 = -com2/(A2*(f2/A2 - (m2*y2)/I2)**2)
        dRFdm2 = (com2*y2)/(I2*(f2/A2 - (m2*y2)/I2)**2)
        dRFdf1 = 0
        dRFdm1 = 0

    dRFdN_ana = np.array([[dRFdf1,dRFdm1,dRFdf2,dRFdm2]])

    return dRFdN_ana

In [ ]:
def analyticalSensitivity_dNdx_V2(N, prop):
    
    '''
    dN/dx for a perticular load case
    '''
    
    x1 = prop['x1']
    x2 = prop['x2']
    x3 = prop['x3']
    E1 = prop['E1']
    E2 = prop['E2']
    
    f1,m1,f2,m2 = N
    
    F = f1+f2
    M = m1+m2
    
    #dN/dx1
    dft_dx1 =  (2*E1*F*np.pi*x1)/(np.pi*E1*(x1**2 - x2**2) + np.pi*E2*x3**2) - (2*E1**2*F*np.pi**2*x1*(x1**2 - x2**2))/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2

    dmt_dx1 = (E1*M*np.pi*x1**3)/((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4) - (E1**2*M*np.pi**2*x1**3*(x1**4 - x2**4))/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2)

    dfb_dx1 = -(2*E1*E2*F*np.pi**2*x1*x3**2)/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2

    dmb_dx1 = -(E1*E2*M*np.pi**2*x1**3*x3**4)/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2)

    dN_dx1 = np.array([[dft_dx1],[dmt_dx1],[dfb_dx1],[dmb_dx1]])
    
    #dN/dx2
    dft_dx2 = (2*E1**2*F*np.pi**2*x2*(x1**2 - x2**2))/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2 - (2*E1*F*np.pi*x2)/(np.pi*E1*(x1**2 - x2**2) + np.pi*E2*x3**2)
    
    dmt_dx2 = (E1**2*M*np.pi**2*x2**3*(x1**4 - x2**4))/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2) - (E1*M*np.pi*x2**3)/((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)
    
    dfb_dx2 = (2*E1*E2*F*np.pi**2*x2*x3**2)/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2
    
    dmb_dx2 = (E1*E2*M*np.pi**2*x2**3*x3**4)/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2)
    
    #dN/dx3

    dft_dx3 = -(2*E1*E2*F*np.pi**2*x3*(x1**2 - x2**2))/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2
    
    dmt_dx3 = -(E1*E2*M*np.pi**2*x3**3*(x1**4 - x2**4))/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2) 
    
    dfb_dx3 = (2*E2*F*np.pi*x3)/(np.pi*E1*(x1**2 - x2**2) + np.pi*E2*x3**2) - (2*E2**2*F*np.pi**2*x3**3)/(E2*np.pi*x3**2 + E1*np.pi*(x1**2 - x2**2))**2
    
    dmb_dx3 = (E2*M*np.pi*x3**3)/((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4) - (E2**2*M*np.pi**2*x3**7)/(4*((E2*np.pi*x3**4)/4 + (E1*np.pi*(x1**4 - x2**4))/4)**2)

    dN_dx1 = np.vstack((dft_dx1,dmt_dx1,dfb_dx1,dmb_dx1))
    dN_dx2 = np.vstack((dft_dx2,dmt_dx2,dfb_dx2,dmb_dx2))
    dN_dx3 = np.vstack((dft_dx3,dmt_dx3,dfb_dx3,dmb_dx3))
    
#     dN_dx = np.hstack((dN_dx1,dN_dx2,dN_dx3))
    return dN_dx1, dN_dx2, dN_dx3


In [ ]:
def checkRF_2D_V3(prop, N):
    
    '''
    
    Returns RF array and min RF and index of the min RF
    
    inputs:
    --------
    
    N: internal loads
    
    '''
    
    A1 = prop['A1']
    A2 = prop['A2']
    y1 = prop['y1']
    y2 = prop['y2']
    I1 = prop['I1']
    I2 = prop['I2']
    ten1 = prop['ten1']
    ten2 = prop['ten2']
    com1 = prop['com1']
    com2 = prop['com2']
    
    RF = []
    ft, mt, fb, mb = N

    #stress in the assembly
    stress_1 = ft/A1 + mt*y1/I1
    stress_1_m = ft/A1 - mt*y1/I1
    stress_2 = fb/A2 + mb*y2/I2
    stress_2_m = fb/A2 - mb*y2/I2

    #Check RF and return the RF index
    RF1 = ten1/(stress_1)
    RF2 = com1/(stress_1)
    RF3 = ten2/(stress_2)
    RF4 = com2/(stress_2)
    RF5 = ten1/(stress_1_m)
    RF6 = com1/(stress_1_m)
    RF7 = ten2/(stress_2_m)
    RF8 = com2/(stress_2_m)

    RF = [RF1,RF2,RF3,RF4,RF5,RF6,RF7,RF8]
    RF =[10e10 if i < 0 else i for i in RF]

    RFmin = min([i for i in RF if i >= 0])

    RF_ind = RF.index(RFmin)

    return RF, RFmin, RF_ind


In [4]:
def sensitivityFromAnalyticalSolution_dRFdU_ana_V3(u, c_cir, prop, L):
    '''
    
    From solving analytical equations Notebook 7 10/10/2017
    RF4=com2/((eN1+fN2/A2)+(gN1+hN2)y2/I2) where e,f,g,h are the values in lk
    corresponding to the internal load cl3 and cl4
    
    
    V3: 09/07/2018
    
    Full rank u matrix for validation
    
    '''
    A1 = prop['A1']
    A2 = prop['A2']
    y1 = prop['y1']
    y2 = prop['y2']
    I1 = prop['I1']
    I2 = prop['I2']
    ten1 = prop['ten1']
    ten2 = prop['ten2']
    com1 = prop['com1']
    com2 = prop['com2']
    
    u1, u2, u3, u4 = u
    
    l11, l12, l13, l14 = L[0,:]
    l21, l22, l23, l24 = L[1,:]
    l31, l32, l33, l34 = L[2,:]
    l41, l42, l43, l44 = L[3,:]
    
    # Analytical solutions
    dRFdU_ana = []

    #RF1
    if c_cir == 0:
        
        dRF1du1_ana = ten1*(-l21*y1/I1 - l11/A1)/(y1*(l21*u1 + l22*u2 + l23*u3 + l24*u4)/I1 + (l11*u1 + l21*u2 + l31*u3 + l41*u4)/A1)**2
        dRF1du2_ana = ten1*(-l22*y1/I1 - l21/A1)/(y1*(l21*u1 + l22*u2 + l23*u3 + l24*u4)/I1 + (l11*u1 + l21*u2 + l31*u3 + l41*u4)/A1)**2
        dRF1du3_ana = ten1*(-l23*y1/I1 - l31/A1)/(y1*(l21*u1 + l22*u2 + l23*u3 + l24*u4)/I1 + (l11*u1 + l21*u2 + l31*u3 + l41*u4)/A1)**2
        dRF1du4_ana = ten1*(-l24*y1/I1 - l41/A1)/(y1*(l21*u1 + l22*u2 + l23*u3 + l24*u4)/I1 + (l11*u1 + l21*u2 + l31*u3 + l41*u4)/A1)**2
        
        dRFdU_ana = [dRF1du1_ana, dRF1du2_ana, dRF1du3_ana, dRF1du4_ana]

    #RF2
    if c_cir == 1:
        
        N1_RF2 , N2_RF2  = gg[:,[0]],gg[:,[1]]
        dRF2du1_ana = -(com1*(a/A1 + (c*y1)/I1))/((N1_RF2*a + N2_RF2*b)/A1 + (y1*(N1_RF2*c + N2_RF2*d))/I1)**2
        dRF2du2_ana = -(com1*(b/A1 + (d*y1)/I1))/((N1_RF2*a + N2_RF2*b)/A1 + (y1*(N1_RF2*c + N2_RF2*d))/I1)**2
        
        dRFdU_ana = [dRF2du1_ana,dRF2du2_ana]

    #RF3
    if c_cir == 2:
        
        N1_RF3 , N2_RF3  = gg[:,[0]],gg[:,[1]]
        dRF3du1_ana = (ten2*(e/A2 + (g*y2)/I2))/((N1_RF3*e + N2_RF3*f)/A2 + (y2*(N1_RF3*g + N2_RF3*h))/I2)**2
        dRF3du2_ana = (ten2*(f/A2 + (h*y2)/I2))/((N1_RF3*e + N2_RF3*f)/A2 + (y2*(N1_RF3*g + N2_RF3*h))/I2)**2
        
        dRFdU_ana = [dRF3du1_ana,dRF3du2_ana]

    #RF4
    if c_cir == 3:
        
        N1_RF4 , N2_RF4  = gg[:,[0]],gg[:,[1]]
        dRF4du1_ana = (com2*(e/A2 + (g*y2)/I2))/((N1_RF4*e + N2_RF4*f)/A2 + (y2*(N1_RF4*g + N2_RF4*h))/I2)**2
        dRF4du2_ana = (com2*(f/A2 + (h*y2)/I2))/((N1_RF4*e + N2_RF4*f)/A2 + (y2*(N1_RF4*g + N2_RF4*h))/I2)**2
        
        dRFdU_ana = [dRF4du1_ana,dRF4du2_ana]

    #RF5
    if c_cir == 4:
        
        dRF5du1_ana = ten1*(l21*y1/I1 - l11/A1)/(-y1*(l21*u1 + l22*u2 + l23*u3 + l24*u4)/I1 + (l11*u1 + l21*u2 + l31*u3 + l41*u4)/A1)**2
        dRF5du2_ana = ten1*(l22*y1/I1 - l21/A1)/(-y1*(l21*u1 + l22*u2 + l23*u3 + l24*u4)/I1 + (l11*u1 + l21*u2 + l31*u3 + l41*u4)/A1)**2
        dRF5du3_ana = ten1*(l23*y1/I1 - l31/A1)/(-y1*(l21*u1 + l22*u2 + l23*u3 + l24*u4)/I1 + (l11*u1 + l21*u2 + l31*u3 + l41*u4)/A1)**2
        dRF5du4_ana = ten1*(l24*y1/I1 - l41/A1)/(-y1*(l21*u1 + l22*u2 + l23*u3 + l24*u4)/I1 + (l11*u1 + l21*u2 + l31*u3 + l41*u4)/A1)**2
        
        dRFdU_ana = [dRF5du1_ana, dRF5du2_ana, dRF5du3_ana, dRF5du4_ana]

    #RF6
    if c_cir == 5:
        
        N1_RF6 , N2_RF6  = gg[:,[0]],gg[:,[1]]
        dRF6du1_ana = -(com1*(a/A1 - (c*y1)/I1))/((N1_RF6*a + N2_RF6*b)/A1 - (y1*(N1_RF6*c + N2_RF6*d))/I1)**2
        dRF6du2_ana = -(com1*(b/A1 - (d*y1)/I1))/((N1_RF6*a + N2_RF6*b)/A1 - (y1*(N1_RF6*c + N2_RF6*d))/I1)**2
        
        dRFdU_ana= [dRF6du1_ana,dRF6du2_ana]

    #RF7
    if c_cir == 6:
        
        N1_RF7 , N2_RF7  = gg[:,[0]],gg[:,[1]]
        dRF7du1_ana = (ten2*(e/A2 - (g*y2)/I2))/((N1_RF7*e + N2_RF7*f)/A2 - (y2*(N1_RF7*g + N2_RF7*h))/I2)**2
        dRF7du2_ana = (ten2*(f/A2 - (h*y2)/I2))/((N1_RF7*e + N2_RF7*f)/A2 - (y2*(N1_RF7*g + N2_RF7*h))/I2)**2
        
        dRFdU_ana = [dRF7du1_ana,dRF7du2_ana]

    #RF8
    if c_cir == 7:
        
        N1_RF8 , N2_RF8  = gg[:,[0]],gg[:,[1]]
        dRF8du1_ana = (com2*(e/A2 - (g*y2)/I2))/((N1_RF8*e + N2_RF8*f)/A2 - (y2*(N1_RF8*g + N2_RF8*h))/I2)**2
        dRF8du2_ana = (com2*(f/A2 - (h*y2)/I2))/((N1_RF8*e + N2_RF8*f)/A2 - (y2*(N1_RF8*g + N2_RF8*h))/I2)**2
        
        dRFdU_ana = [dRF8du1_ana,dRF8du2_ana]
        
    return dRFdU_ana

In [6]:
def compoundBar_compute_dNdNk (n_test, L):

    '''
    Computes dNdNk using finite difference. 
    This should be constant
    
    
    09/07/2018
    '''
    #Finite difference

    #Initial points of the load case
    ft, mt, fb, mb = n_test

    #creating delta loads
    del_ft = ft*np.array([1e-8,0,0,0])
    del_mt = mt*np.array([0,1e-8,0,0])
    del_fb = fb*np.array([0,0,1e-8,0])
    del_mb = mb*np.array([0,0,0,1e-8])

    #perturbed loads
    n_test_del_ft = n_test + del_ft
    n_test_del_mt = n_test + del_mt
    n_test_del_fb = n_test + del_fb
    n_test_del_mb = n_test + del_mb

    u_del_ft = np.dot(n_test_del_ft,np.linalg.pinv(L))
    u_del_mt = np.dot(n_test_del_mt,np.linalg.pinv(L))
    u_del_fb = np.dot(n_test_del_fb,np.linalg.pinv(L))
    u_del_mb = np.dot(n_test_del_mb,np.linalg.pinv(L))

    #ERROR identification
    a, _, _, _ = np.dot(u_del_ft,L)
    _, b, _, _ = np.dot(u_del_mt, L)
    _, _, c, _ = np.dot(u_del_fb, L)
    _, _, _, d = np.dot(u_del_mb, L)

    del_ft_k = n_test_del_ft[0] - a 
    del_mt_k = n_test_del_mt[1] - b
    del_fb_k = n_test_del_fb[2] - c
    del_mb_k = n_test_del_mb[3] - d

    dNdNk = np.array([[del_ft[0]/(del_ft[0]- del_ft_k), 0, 0, 0],
                      [0, del_mt[1]/(del_mt[1]- del_mt_k), 0 , 0],
                      [0, 0, del_fb[2]/(del_fb[2]- del_fb_k), 0],
                      [0, 0, 0, del_mb[3]/(del_mb[3] - del_mb_k)]])
    
    return dNdNk

In [ ]:
def compoundBar_compute_dNdNk_dNdNr (n_test, L, k):

    '''
    Computes dNdNk using finite difference. 
    This should be constant
    
    
    16/07/2018
    
    '''
    #Finite difference

    #Initial points of the load case
    ft, mt, fb, mb = n_test
    
    #Split characteristic loads
    Lk = L[:k,:]
    Lr = L[k:,:]

    #creating delta loads
    del_ft = ft*np.array([1e-8,0,0,0])
    del_mt = mt*np.array([0,1e-8,0,0])
    del_fb = fb*np.array([0,0,1e-8,0])
    del_mb = mb*np.array([0,0,0,1e-8])

    #perturbed loads
    n_test_del_ft = n_test + del_ft
    n_test_del_mt = n_test + del_mt
    n_test_del_fb = n_test + del_fb
    n_test_del_mb = n_test + del_mb

    u_del_ft_k = np.dot(n_test_del_ft,np.linalg.pinv(Lk))
    u_del_mt_k = np.dot(n_test_del_mt,np.linalg.pinv(Lk))
    u_del_fb_k = np.dot(n_test_del_fb,np.linalg.pinv(Lk))
    u_del_mb_k = np.dot(n_test_del_mb,np.linalg.pinv(Lk))

    #ERROR identification
    a_k, _, _, _ = np.dot(u_del_ft_k,Lk)
    _, b_k, _, _ = np.dot(u_del_mt_k, Lk)
    _, _, c_k, _ = np.dot(u_del_fb_k, Lk)
    _, _, _, d_k = np.dot(u_del_mb_k, Lk)

    del_ft_k = n_test_del_ft[0] - a_k
    del_mt_k = n_test_del_mt[1] - b_k
    del_fb_k = n_test_del_fb[2] - c_k
    del_mb_k = n_test_del_mb[3] - d_k

    dNdNk = np.array([[del_ft[0]/(del_ft[0]- del_ft_k), 0, 0, 0],
                      [0, del_mt[1]/(del_mt[1]- del_mt_k), 0 , 0],
                      [0, 0, del_fb[2]/(del_fb[2]- del_fb_k), 0],
                      [0, 0, 0, del_mb[3]/(del_mb[3] - del_mb_k)]])
    
    #r
    u_del_ft_r = np.dot(n_test_del_ft,np.linalg.pinv(Lr))
    u_del_mt_r = np.dot(n_test_del_mt,np.linalg.pinv(Lr))
    u_del_fb_r = np.dot(n_test_del_fb,np.linalg.pinv(Lr))
    u_del_mb_r = np.dot(n_test_del_mb,np.linalg.pinv(Lr))

    #ERROR identification
    a_r, _, _, _ = np.dot(u_del_ft_r,Lr)
    _, b_r, _, _ = np.dot(u_del_mt_r, Lr)
    _, _, c_r, _ = np.dot(u_del_fb_r, Lr)
    _, _, _, d_r = np.dot(u_del_mb_r, Lr)

    del_ft_r = n_test_del_ft[0] - a_r
    del_mt_r = n_test_del_mt[1] - b_r
    del_fb_r = n_test_del_fb[2] - c_r
    del_mb_r = n_test_del_mb[3] - d_r
    
    dNdNr = np.array([[del_ft[0]/(del_ft[0]- del_ft_r), 0, 0, 0],
                  [0, del_mt[1]/(del_mt[1]- del_mt_r), 0 , 0],
                  [0, 0, del_fb[2]/(del_fb[2]- del_fb_r), 0],
                  [0, 0, 0, del_mb[3]/(del_mb[3] - del_mb_r)]])
    
    return dNdNk, dNdNr